# Master code file for IFC

In [3]:
import numpy as np
import pandas as pd

import ifcopenshell

import ifcopenshell.util.element
from ifcopenshell.util.selector import Selector
from ifcopenshell.util.element import get_psets as check

import uuid
import datetime
import random




## Default Functions

In [4]:
def get_psets(element):
    psets = {}
    if element.is_a("IfcTypeObject"):
        if element.HasPropertySets:
            for definition in element.HasPropertySets:
                psets[definition.Name] = get_property_definition(definition)
    elif hasattr(element, "IsDefinedBy"):
        for relationship in element.IsDefinedBy:
            if relationship.is_a("IfcRelDefinesByProperties"):
                definition = relationship.RelatingPropertyDefinition
                psets[definition.Name] = get_property_definition(definition)
    return psets


def get_property_definition(definition):
    if definition is not None:
        props = {}
        if definition.is_a("IfcElementQuantity"):
            props.update(get_quantities(definition.Quantities))
        elif definition.is_a("IfcPropertySet"):
            props.update(get_properties(definition.HasProperties))
        else:
            # Entity introduced in IFC4
            # definition.is_a('IfcPreDefinedPropertySet'):
            for prop in range(4, len(definition)):
                if definition[prop] is not None:
                    props[definition.attribute_name(prop)] = definition[prop]
        return props


def get_quantities(quantities):
    results = {}
    for quantity in quantities:
        if quantity.is_a("IfcPhysicalSimpleQuantity"):
            results[quantity.Name] = quantity[3]
    return results


def get_properties(properties):
    results = {}
    for prop in properties or []:
        if prop.is_a("IfcPropertySingleValue"):
            results[prop.Name] = prop.NominalValue.wrappedValue if prop.NominalValue else None
        elif prop.is_a("IfcComplexProperty"):
            data = {k: v for k, v in prop.get_info().items() if v is not None and k != "Name"}
            data["properties"] = get_properties(prop.HasProperties)
            del data["HasProperties"]
            results[prop.Name] = data
    return results


def get_type(element):
    if element.is_a("IfcTypeObject"):
        return element
    elif hasattr(element, "IsTypedBy") and element.IsTypedBy:
        return element.IsTypedBy[0].RelatingType
    elif hasattr(element, "IsDefinedBy") and element.IsDefinedBy:  # IFC2X3
        for relationship in element.IsDefinedBy:
            if relationship.is_a("IfcRelDefinesByType"):
                return relationship.RelatingType


def get_material(element, should_skip_usage=False):
    if hasattr(element, "HasAssociations") and element.HasAssociations:
        for relationship in element.HasAssociations:
            if relationship.is_a("IfcRelAssociatesMaterial"):
                if should_skip_usage:
                    if relationship.RelatingMaterial.is_a("IfcMaterialLayerSetUsage"):
                        return relationship.RelatingMaterial.ForLayerSet
                    elif relationship.RelatingMaterial.is_a("IfcMaterialProfileSetUsage"):
                        return relationship.RelatingMaterial.ForProfileSet
                return relationship.RelatingMaterial
    relating_type = get_type(element)
    if relating_type != element and hasattr(relating_type, "HasAssociations") and relating_type.HasAssociations:
        return get_material(relating_type, should_skip_usage)


def get_container(element):
    aggregate = get_aggregate(element)
    if aggregate:
        return get_container(aggregate)
    if hasattr(element, "ContainedInStructure") and element.ContainedInStructure:
        return element.ContainedInStructure[0].RelatingStructure


def get_decomposition(element):
    queue = [element]
    results = []
    while queue:
        element = queue.pop()
        for rel in getattr(element, "ContainsElements", []):
            queue.extend(rel.RelatedElements)
            results.extend(rel.RelatedElements)
        for rel in getattr(element, "IsDecomposedBy", []):
            queue.extend(rel.RelatedObjects)
            results.extend(rel.RelatedObjects)
    return results


def get_aggregate(element):
    if hasattr(element, "Decomposes") and element.Decomposes:
        return element.Decomposes[0].RelatingObject


def replace_attribute(element, old, new):
    for i, attribute in enumerate(element):
        if has_element_reference(attribute, old):
            new_attribute = element.walk(lambda v: v == old, lambda v: new, attribute)
            element[i] = new_attribute


def has_element_reference(value, element):
    if isinstance(value, (tuple, list)):
        for v in value:
            if has_element_reference(v, element):
                return True
        return False
    return value == element


def remove_deep(ifc_file, element):
    # @todo maybe some sort of try-finally mechanism.
    ifc_file.batch()
    subgraph = list(ifc_file.traverse(element, breadth_first=True))
    subgraph_set = set(subgraph)
    for ref in subgraph[::-1]:
        if ref.id() and len(set(ifc_file.get_inverse(ref)) - subgraph_set) == 0:
            ifc_file.remove(ref)
    ifc_file.unbatch()


def copy(ifc_file, element):
    new = ifc_file.create_entity(element.is_a())
    for i, attribute in enumerate(element):
        if attribute is None:
            continue
        if new.attribute_name(i) == "GlobalId":
            new[i] = ifcopenshell.guid.new()
        else:
            new[i] = attribute
    return new


def copy_deep(ifc_file, element):
    new = ifc_file.create_entity(element.is_a())
    for i, attribute in enumerate(element):
        if attribute is None:
            continue
        if isinstance(attribute, ifcopenshell.entity_instance):
            attribute = copy_deep(ifc_file, attribute)
        elif isinstance(attribute, tuple) and attribute and isinstance(attribute[0], ifcopenshell.entity_instance):
            attribute = list(attribute)
            for j, item in enumerate(attribute):
                attribute[j] = copy_deep(ifc_file, item)
        new[i] = attribute
    return new

In [5]:
ifc_file= ifcopenshell.open('../IFC_explorer/OMC2-R8-2.ifc')

### Edited function - Get Pset Path

In [ ]:
def get_psets_path(element):
    psets = {}
    if element.is_a("IfcTypeObject"):
        if element.HasPropertySets:
            for definition in element.HasPropertySets:
                psets[definition.Name] = get_property_definition(definition)
                psets[definition.Name+" Path"]="HEhehe"
         
    elif hasattr(element, "IsDefinedBy"):
        a=0
        for relationship in element.IsDefinedBy:
        
            if relationship.is_a("IfcRelDefinesByProperties"):
                definition = relationship.RelatingPropertyDefinition
                psets[definition.Name] = get_property_definition(definition,"element.IsDefinedBy[{}].RelatingPropertyDefinition".format(a))
#                 psets[definition.Name+" Path"]=
            a=a+1
    return psets


def get_property_definition(definition,path):
    if definition is not None:
        props = {}
        if definition.is_a("IfcElementQuantity"):
            props.update(get_quantities(definition.Quantities,path+".Quantities"))
        elif definition.is_a("IfcPropertySet"):
            props.update(get_properties(definition.HasProperties,path+".HasProperties"))
        else:
            # Entity introduced in IFC4
            # definition.is_a('IfcPreDefinedPropertySet'):
            for prop in range(4, len(definition)):
                if definition[prop] is not None:
                    props[definition.attribute_name(prop)] = definition[prop]
        return props


def get_quantities(quantities,path):
    results = {}
    a=0
    for quantity in quantities:
        
        if quantity.is_a("IfcPhysicalSimpleQuantity"):
            results[quantity.Name] = quantity[3]
            results[quantity.Name+" Path"]=path+"["+str(a)+"]"+"[3]"
        a=a+1
    return results


def get_properties(properties,path):
    results = {}
    a=0
    for prop in properties or []:
        if prop.is_a("IfcPropertySingleValue"):
            results[prop.Name] = prop.NominalValue.wrappedValue if prop.NominalValue else None
            results[prop.Name+" Path"]=path+"[{}]".format(a) +"[2][0]"   
        elif prop.is_a("IfcComplexProperty"):
            data = {k: v for k, v in prop.get_info().items() if v is not None and k != "Name"}
            data["properties"] = get_properties(prop.HasProperties)
            del data["HasProperties"]
            results[prop.Name] = data
        a=a+1
    return results


In [ ]:
## This will get you all path of all elements in get_pset

get_psets_path(element)

## Select items

### By Type

In [7]:
ifc_file.by_type('IfcSite')

## Owner History required when creating new realtionship
owner_history = ifc_file.by_type("IfcOwnerHistory")[0]

## Deal with Buildong storey / elevations
BuildingStorey=ifc_file.by_type("IfcBuildingStorey")

## Both 3 works to get IFCPile, etc
products = ifc_file.by_type('IfcProduct')
BuildingElement = ifc_file.by_type('IfcBuildingElement')
ifcelements = ifc_file.by_type("IfcElement")

## Get Types
ifc_site_dataset = ifc_file.by_type('IfcSite')
ifc_buildingelementproxy_dataset = ifc_file.by_type('IfcBuildingElementProxy')
ifc_stair_dataset = ifc_file.by_type('IfcStair')
ifc_column_dataset = ifc_file.by_type('IfcColumn')
ifc_beam_dataset = ifc_file.by_type('IfcBeam')
ifc_openingelement_dataset = ifc_file.by_type('IfcOpeningElement')
ifc_wasteterminal_dataset = ifc_file.by_type('IfcWasteTerminal')
ifc_member_dataset = ifc_file.by_type('IfcMember')
ifc_wall_dataset = ifc_file.by_type('IfcWall')
ifc_slab_dataset = ifc_file.by_type('IfcSlab')
ifc_stairflight_dataset = ifc_file.by_type('IfcStairFlight')

## Super Type/Sub Type (TBD)
super_type = ifc_pile.supertype()
super_type.subtypes()

NameError: name 'ifc_pile' is not defined

### By Others

In [ ]:
## to check if is_a = True/False, also to check is_a WHAT?
products.is_a("IfcTypeObject)
products.is_a()

## By ID ~ #323232 , and by GUID
products=ifc.by_id(1)
products=ifc.by_guid('0EI0MSHbX9gg8Fxwar7lL8')

## Access GUID and name of elemnt
print(wall.GlobalId)
print(wall.Name)              
              

### By Selector

In [ ]:
## Selector
selector=Selector()
              
# This finds all slabs with "Arup" (case-sensitive) in the Name
elements = selector.parse(ifc_file, '.IfcPile[Name *= "Arup"]')

# This finds all slabs which have a quantified net volume greater than 10 units
elements = selector.parse(ifc, '.IfcSlab[Qto_SlabBaseQuantities.NetVolume > "10"]')

# If 0ehnsYoIDA7wC8yu69IDjv is the GlobalId of an IfcBuildingStorey, this gets all of the elements in that storey.
elements = selector.parse(ifc, '@ #0ehnsYoIDA7wC8yu69IDjv')
              
# If 1uVUwUxTX9Jg1NHVw5KZhI is the GlobalId of an IfcTypeElement, this gets all of the elements of that type.
elements = selector.parse(ifc, '* #1uVUwUxTX9Jg1NHVw5KZhI')
              
# This gets all the 2HR fire rated walls in a particular building storey:
elements = selector.parse(ifc, '@ #0ehnsYoIDA7wC8yu69IDjv & .IfcWall[Pset_WallCommon.FireRating = "2HR"]')              
elements =selector.parse(ifc_file, '.IfcWall | .IfcColumn | .IfcBeam | .IfcSlab | .IfcStair  ')

# This gets all walls and slabs in a particular building storey
elements = selector.parse(ifc, '@ #0ehnsYoIDA7wC8yu69IDjv & ( .IfcWall | .IfcSlab )')


![IFC Hiearchy](IFCTree.jpg)

## Breakdown

### Common Element Psets

### TO check property set of an element

In [ ]:
from ifcopenshell.util.element import get_psets as check
check(element)

### To check Z coordiantes of an element

In [ ]:
get_local_placement(each.ObjectPlacement)[0:3,3][2]

### Route to Material and Finishes

In [ ]:
### Route to Material and Finishes
for each in elements:
    for each2 in each.IsDefinedBy:
        print(each2.RelatingPropertyDefinition.Name)
        if "Materials and Finishes" not in list(each2.RelatingPropertyDefinition):
            pass
        elif "Concrete Grade" not in list(each2.RelatingPropertyDefinition.HasProperties):

### Route to Materials and Finished - Related Material
for each in elements:
    for each2 in each.HasAssociations:
        print(each2.HasAssociations.Name)
        print(each2.HasAssociations.RelatingMaterial.ForLayerSet.MaterialLayers)
        

### Route to IFC Export Type

In [ ]:
for each in elements:
    for each2 in each.IsTypedBy[0].RelatingType.HasPropertySets:
        for each3 in each2.HasProperties:
            if each3.Name=="IfcExportType":